In [67]:
import xlrd
import os
from pandas import Series,DataFrame
import numpy as np
import pandas as pd
import datetime

#### 1、通过header=None的方式读取excel

In [21]:
file_name='data/convert_2.xlsx'
read_data=pd.read_excel(file_name,sheetname=0,header=None)

In [22]:
read_data.head()

,0,1,2,3,4,5,6
0,年 月 日,风暴潮组,海浪组,海浪组,海浪组,海冰组,海冰组
1,NaN,主班,主班,副班,警报班,主班,主班
2,2017-12-01 00:00:00,刘秋兴,韦锋余,徐瑞,王娟娟,唐茂宁,刘琼
3,2017-12-02 00:00:00,刘秋兴,邢闯,韦锋余,徐瑞,唐茂宁,刘琼
4,2017-12-03 00:00:00,刘秋兴,侯放,邢闯,韦锋余,唐茂宁,刘琼


In [23]:
test_data=pd.DataFrame(np.arange(6).reshape(2,3))
test_data

,0,1,2
0,0,1,2
1,3,4,5


* 读取行标题

In [24]:
read_data.columns.tolist()

[0, 1, 2, 3, 4, 5, 6]

#### 1.1 去掉第一列，取剩下的前两行

In [25]:
# 会出错 KeyError: 1
# read_data[1]
read_data[:2]

,0,1,2,3,4,5,6
0,年 月 日,风暴潮组,海浪组,海浪组,海浪组,海冰组,海冰组
1,NaN,主班,主班,副班,警报班,主班,主班


#### 1.2获取第一列起始至结束；第0行起始至第一行的数据

In [26]:
columns_names=read_data.iloc[0:2,1:]
columns_names

,1,2,3,4,5,6
0,风暴潮组,海浪组,海浪组,海浪组,海冰组,海冰组
1,主班,主班,副班,警报班,主班,主班


In [27]:
columns_names.shape[1]

6

In [28]:
np.arange(columns_names.shape[1])

array([0, 1, 2, 3, 4, 5])

#### 1.3 修改columns

In [29]:
columns_names.columns=np.arange(columns_names.shape[1])

In [30]:
columns_names

,0,1,2,3,4,5
0,风暴潮组,海浪组,海浪组,海浪组,海冰组,海冰组
1,主班,主班,副班,警报班,主班,主班


* Nan不能通过==np.nan的方式判断是否为空
* 只能通过is np.nan的方式

#### 1.4 判断表头是否有nan值

In [31]:
index_null=np.where(columns_names.isnull())
index_null

(array([], dtype=int64), array([], dtype=int64))

In [37]:
# 注意使用此种方法永远都是2
len(index_null)

2

* 判断dataframe是否有nan值的方式

In [36]:
columns_names.isnull().values.sum()

0

In [40]:
index_validation=columns_names.isnull().values.any()

In [41]:
if index_validation:
    print("存在null值，不符合条件")

### 2 使用where找到isnull的位置

* dataframe索引 df[val]：选取df的单个列或一组列

In [32]:
columns_names[index_columns][index_row]

nan

* 对于dataframe使用索引的方式第一个值是列（列的起始值是从0开始）；第二个值是行号（行号是从0开始）

In [129]:
print(columns_names[4][1])
print(columns_names[6][1])
print(columns_names[1][1])

警报班
nan
nan


#### 3 获取人员主体dataframe

In [49]:
read_data[2:-1].head()

,0,1,2,3,4,5,6
2,2017-12-01 00:00:00,刘秋兴,韦锋余,徐瑞,王娟娟,唐茂宁,刘琼
3,2017-12-02 00:00:00,刘秋兴,邢闯,韦锋余,徐瑞,唐茂宁,刘琼
4,2017-12-03 00:00:00,刘秋兴,侯放,邢闯,韦锋余,唐茂宁,刘琼
5,2017-12-04 00:00:00,梁森栋,高志一,侯放,邢闯,唐茂宁,刘琼
6,2017-12-05 00:00:00,梁森栋,王久珂,高志一,侯放,唐茂宁,刘琼


#### 3.1 获取时间索引

In [51]:
read_data[2:-1][0].head()

2    2017-12-01 00:00:00
3    2017-12-02 00:00:00
4    2017-12-03 00:00:00
5    2017-12-04 00:00:00
6    2017-12-05 00:00:00
Name: 0, dtype: object

#### 3.2 重新设置第一列为索引  
使用df..set_index('列名')

In [55]:
convert_data=read_data[2:-1].set_index(0)

In [56]:
convert_data.head()

,1,2,3,4,5,6
0,,,,,,
2017-12-01,刘秋兴,韦锋余,徐瑞,王娟娟,唐茂宁,刘琼
2017-12-02,刘秋兴,邢闯,韦锋余,徐瑞,唐茂宁,刘琼
2017-12-03,刘秋兴,侯放,邢闯,韦锋余,唐茂宁,刘琼
2017-12-04,梁森栋,高志一,侯放,邢闯,唐茂宁,刘琼
2017-12-05,梁森栋,王久珂,高志一,侯放,唐茂宁,刘琼


* 注意使用set_index会默认将原来的列名（eg：0）作为新的索引的name  
所以若想删除0这一行，使用drop的方式不可行，会报错

In [59]:
convert_data.drop(0)

ValueError: labels [0] not contained in axis

In [60]:
convert_data.index

DatetimeIndex(['2017-12-01', '2017-12-02', '2017-12-03', '2017-12-04',
               '2017-12-05', '2017-12-06', '2017-12-07', '2017-12-08',
               '2017-12-09', '2017-12-10', '2017-12-11', '2017-12-12',
               '2017-12-13', '2017-12-14', '2017-12-15', '2017-12-16',
               '2017-12-17', '2017-12-18', '2017-12-19', '2017-12-20',
               '2017-12-21', '2017-12-22', '2017-12-23', '2017-12-24',
               '2017-12-25', '2017-12-26', '2017-12-27', '2017-12-28',
               '2017-12-29', '2017-12-30'],
              dtype='datetime64[ns]', name=0, freq=None)

* 但这个name貌似对检索并无影响

#### 3.3 根据时间索引取值（测试用）

In [64]:
convert_data[:1].shape

(1, 6)

In [65]:
convert_data[:1]

,1,2,3,4,5,6
0,,,,,,
2017-12-01,刘秋兴,韦锋余,徐瑞,王娟娟,唐茂宁,刘琼


In [69]:
datetime.datetime(2017,12,1)

datetime.datetime(2017, 12, 1, 0, 0)

* 注意从dataframe中取出一行就是一个series

In [71]:
type(convert_data.loc[datetime.datetime(2017,12,1)])

pandas.core.series.Series

In [72]:
convert_data.loc[datetime.datetime(2017,12,1)]

1    刘秋兴
2    韦锋余
3     徐瑞
4    王娟娟
5    唐茂宁
6     刘琼
Name: 2017-12-01 00:00:00, dtype: object

#### 3.4 尝试去掉索引列的name 0（测试用）

In [76]:
convert_data[0].tolist()

KeyError: 0

### 4、将columns修改回去

In [74]:
columns_names

,0,1,2,3,4,5
0,风暴潮组,海浪组,海浪组,海浪组,海冰组,海冰组
1,主班,主班,副班,警报班,主班,主班


* 若直接将columns_names（是一个dataframe）  
直接赋值给要修改的dataframe的columns会出错：  
ValueError: Length mismatch: Expected axis has 6 elements, new values have 2 elements

In [73]:
convert_data.columns=columns_names

ValueError: Length mismatch: Expected axis has 6 elements, new values have 2 elements